In [11]:
import cv2
import numpy as np

In [ ]:
# load video path: sample_videos\Autonomous driving lane detection sample video 1.mp4

video_path = "sample_videos\Autonomous driving lane detection sample video 1.mp4"
cap = cv2.VideoCapture(video_path)


# change image frame to gray scale
def gray_scale(frame):
    return cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# apply gaussian blur to the image
def gaussian_blur(frame):
    return cv2.GaussianBlur(frame, (5, 5), 0)

# apply canny edge detection to the image
def canny_edge(frame):
    return cv2.Canny(frame, 50, 150)

# Apply a trapezoidal mask to the frame
def region_of_interest(frame):
    height, width = frame.shape[:2]
    mask = np.zeros_like(frame)

    # Define a trapezoid region of interest
    polygon = np.array([
        [
            (int(width * 0.2), height), # bottom left
            (int(width * 0.8), height), # bottom right
            (int(width * 0.65), int(height * 0.57)), # top right
            (int(width * 0.45), int(height * 0.57)) # top left
        ]
    ], dtype=np.int32)

    # Fill the mask with the polygon
    cv2.fillPoly(mask, polygon, 255)

    # Apply the mask to the frame
    masked_frame = cv2.bitwise_and(frame, mask)
    return masked_frame

import cv2
import numpy as np

video_path = "sample_videos\Autonomous driving lane detection sample video 1.mp4"
cap = cv2.VideoCapture(video_path)

# Change image frame to gray scale
def gray_scale(frame):
    return cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur to the image
def gaussian_blur(frame):
    return cv2.GaussianBlur(frame, (5, 5), 0)

# Apply Canny edge detection to the image
def canny_edge(frame):
    return cv2.Canny(frame, 20, 40)

# Apply a trapezoidal mask to the frame
def region_of_interest(frame):
    height, width = frame.shape[:2]
    mask = np.zeros_like(frame)

    # Define a trapezoid region of interest
    polygon = np.array([
        [
            (int(width * 0.2), height),
            (int(width * 0.8), height),
            (int(width * 0.55), int(height * 0.6)),
            (int(width * 0.45), int(height * 0.6))
        ]
    ], dtype=np.int32)

    # Fill the mask with the polygon
    cv2.fillPoly(mask, polygon, 255)

    # Apply the mask to the frame
    masked_frame = cv2.bitwise_and(frame, mask)
    return masked_frame

# Apply Hough transform to detect lines and extend them
def hough_transform(frame, original_frame):
    lines = cv2.HoughLinesP(
        frame,
        rho=1,
        theta=np.pi / 180,
        threshold=50,
        minLineLength=40,
        maxLineGap=150
    )
    line_frame = np.zeros_like(original_frame)

    left_lines = []
    right_lines = []
    height, width = frame.shape[:2]
    
    # Separate lines into left and right based on slope
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            slope = (y2 - y1) / (x2 - x1 + 1e-6)  # Avoid division by zero
            if slope < 0:  # Left lane
                left_lines.append((x1, y1, x2, y2))
            elif slope > 0:  # Right lane
                right_lines.append((x1, y1, x2, y2))

    # Function to average and extend lines
    def average_and_extend(lines):
        if len(lines) == 0:
            return None
        x_coords = []
        y_coords = []
        for x1, y1, x2, y2 in lines:
            x_coords += [x1, x2]
            y_coords += [y1, y2]
        poly = np.polyfit(y_coords, x_coords, 1)  # Fit a line
        y1 = height
        y2 = int(height * 0.6)
        x1 = int(poly[0] * y1 + poly[1])
        x2 = int(poly[0] * y2 + poly[1])
        return (x1, y1, x2, y2)

    left_lane = average_and_extend(left_lines)
    right_lane = average_and_extend(right_lines)

    # Draw the lanes
    if left_lane is not None:
        x1, y1, x2, y2 = left_lane
        cv2.line(line_frame, (x1, y1), (x2, y2), (255, 0, 0), 10)
    if right_lane is not None:
        x1, y1, x2, y2 = right_lane
        cv2.line(line_frame, (x1, y1), (x2, y2), (255, 0, 0), 10)

    return line_frame

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # convert to gray scale
    gray = gray_scale(frame)

    # apply gaussian blur
    blur = gaussian_blur(gray)

    # apply canny edge detection
    edge = canny_edge(blur)

    # Apply the region of interest mask
    masked_edge = region_of_interest(edge)

    # Apply Hough transform to detect lane lines
    lines = hough_transform(masked_edge, frame)

    # Show the video
    cv2.imshow("Lane Lines", lines)


    # video 60 fps, wait according to fps
    cv2.waitKey(int(1000/60))

    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break



